In [4]:
! pip install selenium

  Using cached selenium-4.29.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.5 MB 857.1 kB/s eta 0:00:11
   --- ------------------------------------ 0.8/9.5 MB 930.9 kB/s eta 0:00:10
   ---- ----------------------------------- 1.0/9.5 MB 963.4 kB/s eta 0:00:09
   ---- ----------------------------------- 1.0/9.5 MB 963.4 kB/s eta 0:00:09
   ----- ---------------------------------- 1.3/9.5 MB 933.5 kB/s eta 0:00:09
   ---

In [5]:
#for action movie - 1296
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action')
driver.maximize_window()
time.sleep(2)

scrolling = True #initialize
while scrolling:
    try:
        # Locate the more_button
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        time.sleep(2)
        
        # Scroll to Locate the more_button
        actions = ActionChains(driver)
        time.sleep(3)
        actions.move_to_element(more_button).perform()#it act as a curser
        time.sleep(5)
        
        # Click the button
        more_button.click()
        time.sleep(3)
        
    except:
        scrolling = False
        break



#create a list to append the data
movie_name = []
voting = []
rating = []
duration = []

#movies xpath
action_movies = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')


for action_movie in action_movies:
   try:
       movie_names = action_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
       movie_votings = action_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
       movie_ratings = action_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
       movie_duration = action_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text                                                  
       
       #append the data into list
       movie_name.append(movie_names)
       voting.append(movie_votings)
       rating.append(movie_ratings)
       duration.append(movie_duration)
   except:
       continue


#create a dataframe
action_movie_df = pd.DataFrame({
    'Movie_name':movie_name,
    'Voting':voting,
    'Rating': rating,
    'Duration':duration
})

driver.quit()

action_movie_df


,Movie_name,Voting,Rating,Duration
0,1. Dune: Part Two,(611K),8.5,2h 46m
1,2. Venom: The Last Dance,(106K),6.0,1h 50m
2,3. Gladiator II,(215K),6.6,2h 28m
3,4. Sonic the Hedgehog 3,(52K),6.9,1h 50m
4,5. The Lord of the Rings: The War of the Rohirrim,(27K),6.3,2h 14m
...,...,...,...,...
784,1169. Jefa De Jefas,(9),8.2,1h 22m
785,1171. The End of Hen,(9),9.7,1h 7m
786,1180. Bush Knife,(7),7.0,1h 30m
787,1182. Kyle and the Last Emerald 2,(7),8.4,15m


In [6]:
#for data cleaning

# remove integer and dot(movie_name)
import re #regular expression
action_movie_df['Movie_name']= action_movie_df['Movie_name'].str.replace(r'^\d+\.\s*', '', regex = True)

#clean the Voting column
def clean_voting(vote):
    return vote.replace('(', '').replace(')', '').replace('K', '000')

# Apply to Voting column
action_movie_df['Voting'] = action_movie_df['Voting'].apply(clean_voting)

#clean the duration:
def convert_to_seconds(duration): # 2h 7m
    hours = 0
    minutes = 0
    splitt = duration.split() #['2h','7m']
    for splitts in splitt:
        if 'h' in splitts:
            hours = int(splitts.replace('h', '')) #[2 ,'7m']
        if 'm' in splitts:
            minutes = int(splitts.replace('m', '')) # [2 , 7]

    return hours* 3600 + minutes * 60

# Apply to duration column
action_movie_df['Duration'] = action_movie_df['Duration'].apply(convert_to_seconds) 

action_movie_df



,Movie_name,Voting,Rating,Duration
0,Dune: Part Two,611000,8.5,9960
1,Venom: The Last Dance,106000,6.0,6600
2,Gladiator II,215000,6.6,8880
3,Sonic the Hedgehog 3,52000,6.9,6600
4,The Lord of the Rings: The War of the Rohirrim,27000,6.3,8040
...,...,...,...,...
784,Jefa De Jefas,9,8.2,4920
785,The End of Hen,9,9.7,4020
786,Bush Knife,7,7.0,5400
787,Kyle and the Last Emerald 2,7,8.4,900


In [7]:
#add a new column Gener
action_movie_df['Gener'] = 'Action'

In [8]:
action_movie_df

,Movie_name,Voting,Rating,Duration,Gener
0,Dune: Part Two,611000,8.5,9960,Action
1,Venom: The Last Dance,106000,6.0,6600,Action
2,Gladiator II,215000,6.6,8880,Action
3,Sonic the Hedgehog 3,52000,6.9,6600,Action
4,The Lord of the Rings: The War of the Rohirrim,27000,6.3,8040,Action
...,...,...,...,...,...
784,Jefa De Jefas,9,8.2,4920,Action
785,The End of Hen,9,9.7,4020,Action
786,Bush Knife,7,7.0,5400,Action
787,Kyle and the Last Emerald 2,7,8.4,900,Action


In [9]:
#war movie 127
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd


driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=war')
driver.maximize_window()
time.sleep(2)

scrolling = True #initialize
while scrolling:
    try:
        # Locate the more_button
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        time.sleep(2)
        
        # Scroll to Locate the more_button
        actions = ActionChains(driver)
        time.sleep(3)
        actions.move_to_element(more_button).perform()
        time.sleep(5)
        
        # Click the button
        more_button.click()
        time.sleep(3)
        
        
    except:
        pass
        scrolling = False


movie_name = []
voting = []
rating = []
duration = []

war_movies = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

    #print(i.text)
for war_movie in war_movies:
   try:
       movie_names = war_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
       movie_votings = war_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
       movie_ratings = war_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
       movie_duration = war_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text                                                  
       
       #append the data into list
       movie_name.append(movie_names)
       voting.append(movie_votings)
       rating.append(movie_ratings)
       duration.append(movie_duration)
   except:
       pass


#create a dataframe
war_movie_df = pd.DataFrame({
    'Movie_name':movie_name,
    'Voting':voting,
    'Rating': rating,
    'Duration':duration
})



driver.quit()
war_movie_df



,Movie_name,Voting,Rating,Duration
0,1. William Tell,(1.6K),5.8,2h 13m
1,2. The Ministry of Ungentlemanly Warfare,(131K),6.8,2h 2m
2,3. The Six Triple Eight,(25K),6.7,2h 7m
3,4. My Dead Friend Zoe,(1K),7.3,1h 43m
4,5. Nr. 24,(14K),7.5,1h 51m
...,...,...,...,...
73,110. Sem chyornykh bumag,(55),6.6,1h 36m
74,112. Krasnye lenty,(18),5.3,1h 2m
75,"114. Por ti, Portugal, eu juro!",(5),7.6,1h 38m
76,118. Dosije Kosovo - Pogrom,(5),7.6,1h 49m


In [10]:
#for data cleaning

# remove integer and dot(movie_name)
import re #regular expression
war_movie_df['Movie_name']= war_movie_df['Movie_name'].str.replace(r'^\d+\.\s*', '', regex = True)

#clean the Voting column
def clean_voting(vote):
    return vote.replace('(', '').replace(')', '').replace('K', '000')

# Apply to Voting column
war_movie_df['Voting'] = war_movie_df['Voting'].apply(clean_voting) 

#clean the duration:
def convert_to_seconds(duration): # 2h 7m
    hours = 0
    minutes = 0
    splitt = duration.split() #['2h','7m']
    for splitts in splitt:
        if 'h' in splitts:
            hours = int(splitts.replace('h', '')) #[2 ,'7m']
        if 'm' in splitts:
            minutes = int(splitts.replace('m', '')) # [2 , 7]

    return hours* 3600 + minutes * 60

# Apply to duration column
war_movie_df['Duration'] = war_movie_df['Duration'].apply(convert_to_seconds) 

war_movie_df


,Movie_name,Voting,Rating,Duration
0,William Tell,1.6000,5.8,7980
1,The Ministry of Ungentlemanly Warfare,131000,6.8,7320
2,The Six Triple Eight,25000,6.7,7620
3,My Dead Friend Zoe,1000,7.3,6180
4,Nr. 24,14000,7.5,6660
...,...,...,...,...
73,Sem chyornykh bumag,55,6.6,5760
74,Krasnye lenty,18,5.3,3720
75,"Por ti, Portugal, eu juro!",5,7.6,5880
76,Dosije Kosovo - Pogrom,5,7.6,6540


In [11]:
#add a new column Gener
war_movie_df['Gener'] = 'War'

In [12]:
war_movie_df

,Movie_name,Voting,Rating,Duration,Gener
0,William Tell,1.6000,5.8,7980,War
1,The Ministry of Ungentlemanly Warfare,131000,6.8,7320,War
2,The Six Triple Eight,25000,6.7,7620,War
3,My Dead Friend Zoe,1000,7.3,6180,War
4,Nr. 24,14000,7.5,6660,War
...,...,...,...,...,...
73,Sem chyornykh bumag,55,6.6,5760,War
74,Krasnye lenty,18,5.3,3720,War
75,"Por ti, Portugal, eu juro!",5,7.6,5880,War
76,Dosije Kosovo - Pogrom,5,7.6,6540,War


In [16]:
#for biography movie - 440
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?title_type=feature&genres=biography&release_date=2024-01-01,2024-12-31')
driver.maximize_window()
time.sleep(2)

scrolling = True #initialize
while scrolling:
    try:
        # Locate the more_button
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        time.sleep(2)
        
        # Scroll to Locate the more_button
        actions = ActionChains(driver)
        time.sleep(3)
        actions.move_to_element(more_button).perform()
        time.sleep(5)
        
        # Click the button
        more_button.click()
        time.sleep(3)
        
        
    except:
        scrolling = False


movie_name = []
voting = []
rating = []
duration = []

biography_movies = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')


for biography_movie in biography_movies:
   try:                                           
       movie_names = biography_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
       movie_votings = biography_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
       movie_ratings = biography_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
       movie_duration = biography_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text                                                  
       
       #append the data into list
       movie_name.append(movie_names)
       voting.append(movie_votings)
       rating.append(movie_ratings)
       duration.append(movie_duration)
   except:
    continue
       


#create a dataframe
biography_movie_df = pd.DataFrame({
    'Movie_name':movie_name,
    'Voting':voting,
    'Rating': rating,
    'Duration':duration
})

driver.quit()
biography_movie_df

,Movie_name,Voting,Rating,Duration
0,1. A Complete Unknown,(58K),7.4,2h 21m
1,2. I'm Still Here,(93K),8.4,2h 17m
2,3. The Apprentice,(49K),7.1,2h 2m
3,4. Queen of the Ring,(744),7.0,2h 20m
4,5. William Tell,(1.6K),5.8,2h 13m
...,...,...,...,...
254,410. Toquinho: Encontros e um Violão,(5),8.6,1h 30m
255,413. ¡Qué suerte hemos tenido!,(16),7.5,1h 2m
256,"416. Georges Pompidou, la cruauté du pouvoir",(6),6.2,1h 44m
257,417. Beyaz Zenciler: Yapim Asamasi,(5),8.6,55m


In [17]:
#for data cleaning

# remove integer and dot(movie_name)
import re #regular expression
biography_movie_df['Movie_name']= biography_movie_df['Movie_name'].str.replace(r'^\d+\.\s*', '', regex = True)

#clean the Voting column
def clean_voting(vote):
    return vote.replace('(', '').replace(')', '').replace('K', '000')

# Apply to Voting column
biography_movie_df['Voting'] = biography_movie_df['Voting'].apply(clean_voting) 

#clean the duration:
def convert_to_seconds(duration): # 2h 7m
    hours = 0
    minutes = 0
    splitt = duration.split() #['2h','7m']
    for splitts in splitt:
        if 'h' in splitts:
            hours = int(splitts.replace('h', '')) #[2 ,'7m']
        if 'm' in splitts:
            minutes = int(splitts.replace('m', '')) # [2 , 7]

    return hours* 3600 + minutes * 60

# Apply to duration column
biography_movie_df['Duration'] = biography_movie_df['Duration'].apply(convert_to_seconds) 

biography_movie_df


,Movie_name,Voting,Rating,Duration
0,A Complete Unknown,58000,7.4,8460
1,I'm Still Here,93000,8.4,8220
2,The Apprentice,49000,7.1,7320
3,Queen of the Ring,744,7.0,8400
4,William Tell,1.6000,5.8,7980
...,...,...,...,...
254,Toquinho: Encontros e um Violão,5,8.6,5400
255,¡Qué suerte hemos tenido!,16,7.5,3720
256,"Georges Pompidou, la cruauté du pouvoir",6,6.2,6240
257,Beyaz Zenciler: Yapim Asamasi,5,8.6,3300


In [18]:
#add a new column Gener
biography_movie_df['Gener'] = 'Biography'

In [19]:
biography_movie_df

,Movie_name,Voting,Rating,Duration,Gener
0,A Complete Unknown,58000,7.4,8460,Biography
1,I'm Still Here,93000,8.4,8220,Biography
2,The Apprentice,49000,7.1,7320,Biography
3,Queen of the Ring,744,7.0,8400,Biography
4,William Tell,1.6000,5.8,7980,Biography
...,...,...,...,...,...
254,Toquinho: Encontros e um Violão,5,8.6,5400,Biography
255,¡Qué suerte hemos tenido!,16,7.5,3720,Biography
256,"Georges Pompidou, la cruauté du pouvoir",6,6.2,6240,Biography
257,Beyaz Zenciler: Yapim Asamasi,5,8.6,3300,Biography


In [21]:
#animation movie 393
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=animation')
driver.maximize_window()
time.sleep(2)

scrolling = True #initialize
while scrolling:
    try:
        # Locate the more_button
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        time.sleep(2)
        
        # Scroll to Locate the more_button
        actions = ActionChains(driver)
        time.sleep(3)
        actions.move_to_element(more_button).perform()
        time.sleep(5)
        
        # Click the button
        more_button.click()
        time.sleep(3)
        
        
    except:
        scrolling = False


movie_name = []
voting = []
rating = []
duration = []

animation_movies = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')


for animation_movie in animation_movies:
   try:
       movie_names = animation_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
       movie_votings = animation_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
       movie_ratings = animation_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
       movie_duration = animation_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text                                                  
       
       #append the data into list
       movie_name.append(movie_names)
       voting.append(movie_votings)
       rating.append(movie_ratings)
       duration.append(movie_duration)
   except:
       continue


#create a dataframe
animation_movie_df = pd.DataFrame({
    'Movie_name':movie_name,
    'Voting':voting,
    'Rating': rating,
    'Duration':duration
})

#animation_movie_df.to_csv('animation_movie.csv',index=False)

#driver.quit()
animation_movie_df

,Movie_name,Voting,Rating,Duration
0,1. Flow,(57K),7.9,1h 25m
1,2. The Wild Robot,(151K),8.2,1h 42m
2,3. The Lord of the Rings: The War of the Rohirrim,(27K),6.3,2h 14m
3,4. Mufasa: The Lion King,(47K),6.6,1h 58m
4,5. Despicable Me 4,(62K),6.2,1h 34m
...,...,...,...,...
214,"287. Hey, Tete",(9),6.4,2h 31m
215,301. Skazochnyy patrul. Shou prodolzhayetsya,(11),7.2,1h 5m
216,309. Let's Go! Anpanman: Baikinman and Lulun,(7),6.5,1h 4m
217,321. Bie ba zuo wen dang hui shi er,(10),7.2,1h 29m


In [22]:
#for data cleaning

# remove integer and dot(movie_name)
import re #regular expression
animation_movie_df['Movie_name']= animation_movie_df['Movie_name'].str.replace(r'^\d+\.\s*', '', regex = True)

#clean the Voting column
def clean_voting(vote):
    return vote.replace('(', '').replace(')', '').replace('K', '000')

# Apply to Voting column
animation_movie_df['Voting'] = animation_movie_df['Voting'].apply(clean_voting) 

#clean the duration:
def convert_to_seconds(duration): # 2h 7m
    hours = 0
    minutes = 0
    splitt = duration.split() #['2h','7m']
    for splitts in splitt:
        if 'h' in splitts:
            hours = int(splitts.replace('h', '')) #[2 ,'7m']
        if 'm' in splitts:
            minutes = int(splitts.replace('m', '')) # [2 , 7]

    return hours* 3600 + minutes * 60

# Apply to duration column
animation_movie_df['Duration'] = animation_movie_df['Duration'].apply(convert_to_seconds) 

animation_movie_df


,Movie_name,Voting,Rating,Duration
0,Flow,57000,7.9,5100
1,The Wild Robot,151000,8.2,6120
2,The Lord of the Rings: The War of the Rohirrim,27000,6.3,8040
3,Mufasa: The Lion King,47000,6.6,7080
4,Despicable Me 4,62000,6.2,5640
...,...,...,...,...
214,"Hey, Tete",9,6.4,9060
215,Skazochnyy patrul. Shou prodolzhayetsya,11,7.2,3900
216,Let's Go! Anpanman: Baikinman and Lulun,7,6.5,3840
217,Bie ba zuo wen dang hui shi er,10,7.2,5340


In [23]:
#add a new column Gener
animation_movie_df['Gener'] = 'Animation'

In [24]:
animation_movie_df

,Movie_name,Voting,Rating,Duration,Gener
0,Flow,57000,7.9,5100,Animation
1,The Wild Robot,151000,8.2,6120,Animation
2,The Lord of the Rings: The War of the Rohirrim,27000,6.3,8040,Animation
3,Mufasa: The Lion King,47000,6.6,7080,Animation
4,Despicable Me 4,62000,6.2,5640,Animation
...,...,...,...,...,...
214,"Hey, Tete",9,6.4,9060,Animation
215,Skazochnyy patrul. Shou prodolzhayetsya,11,7.2,3900,Animation
216,Let's Go! Anpanman: Baikinman and Lulun,7,6.5,3840,Animation
217,Bie ba zuo wen dang hui shi er,10,7.2,5340,Animation


In [26]:
#adventure movie - 577
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=adventure')
driver.maximize_window()
time.sleep(2)

scrolling = True #initialize
while scrolling:
    try:
        # Locate the more_button
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        time.sleep(2)
        
        # Scroll to Locate the more_button
        actions = ActionChains(driver)
        time.sleep(3)
        actions.move_to_element(more_button).perform()
        time.sleep(5)
        
        # Click the button
        more_button.click()
        time.sleep(3)
        
        
    except:
        scrolling = False


movie_name = []
voting = []
rating = []
duration = []

adventure_movies = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

    #print(i.text)
for adventure_movie in adventure_movies:
   try:
       movie_names = adventure_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
       movie_votings = adventure_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
       movie_ratings = adventure_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
       movie_duration = adventure_movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text                                                  
       
       #append the data into list
       movie_name.append(movie_names)
       voting.append(movie_votings)
       rating.append(movie_ratings)
       duration.append(movie_duration)
   except:
       continue


#create a dataframe
adventure_movie_df = pd.DataFrame({
    'Movie_name':movie_name,
    'Voting':voting,
    'Rating': rating,
    'Duration':duration
})
driver.quit()
adventure_movie_df

,Movie_name,Voting,Rating,Duration
0,1. Flow,(57K),7.9,1h 25m
1,2. Dune: Part Two,(611K),8.5,2h 46m
2,3. Venom: The Last Dance,(107K),6.0,1h 50m
3,4. Gladiator II,(215K),6.6,2h 28m
4,5. Sonic the Hedgehog 3,(52K),6.9,1h 50m
...,...,...,...,...
378,545. Elli and the Ghostly Ghost Train,(88),5.0,1h 26m
379,546. The Kingfish 2,(10),7.9,1h 12m
380,553. Lajjpal,(8),9.0,4h 12m
381,567. Beyaz Zenciler: Yapim Asamasi,(5),8.6,55m


In [27]:
#for data cleaning

# remove integer and dot(movie_name)
import re #regular expression
adventure_movie_df['Movie_name']= adventure_movie_df['Movie_name'].str.replace(r'^\d+\.\s*', '', regex = True)

#clean the Voting column
def clean_voting(vote):
    return vote.replace('(', '').replace(')', '').replace('K', '000')

# Apply to Voting column
adventure_movie_df['Voting'] = adventure_movie_df['Voting'].apply(clean_voting) 

#clean the duration:
def convert_to_seconds(duration): # 2h 7m
    hours = 0
    minutes = 0
    splitt = duration.split() #['2h','7m']
    for splitts in splitt:
        if 'h' in splitts:
            hours = int(splitts.replace('h', '')) #[2 ,'7m']
        if 'm' in splitts:
            minutes = int(splitts.replace('m', '')) # [2 , 7]

    return hours* 3600 + minutes * 60 #[2*3600 + 7*60]

# Apply to duration column
adventure_movie_df['Duration'] = adventure_movie_df['Duration'].apply(convert_to_seconds) 

adventure_movie_df


,Movie_name,Voting,Rating,Duration
0,Flow,57000,7.9,5100
1,Dune: Part Two,611000,8.5,9960
2,Venom: The Last Dance,107000,6.0,6600
3,Gladiator II,215000,6.6,8880
4,Sonic the Hedgehog 3,52000,6.9,6600
...,...,...,...,...
378,Elli and the Ghostly Ghost Train,88,5.0,5160
379,The Kingfish 2,10,7.9,4320
380,Lajjpal,8,9.0,15120
381,Beyaz Zenciler: Yapim Asamasi,5,8.6,3300


In [28]:
#add a new column Gener
adventure_movie_df['Gener'] = 'Adventure'

In [29]:
adventure_movie_df

,Movie_name,Voting,Rating,Duration,Gener
0,Flow,57000,7.9,5100,Adventure
1,Dune: Part Two,611000,8.5,9960,Adventure
2,Venom: The Last Dance,107000,6.0,6600,Adventure
3,Gladiator II,215000,6.6,8880,Adventure
4,Sonic the Hedgehog 3,52000,6.9,6600,Adventure
...,...,...,...,...,...
378,Elli and the Ghostly Ghost Train,88,5.0,5160,Adventure
379,The Kingfish 2,10,7.9,4320,Adventure
380,Lajjpal,8,9.0,15120,Adventure
381,Beyaz Zenciler: Yapim Asamasi,5,8.6,3300,Adventure


In [30]:
#convert df to csv
war_movie_df.to_csv("war_movies_df.csv", index=False)

In [31]:
action_movie_df.to_csv("action_movies.csv", index=False)

In [32]:
biography_movie_df.to_csv("biography_movies.csv", index=False)

In [33]:
animation_movie_df.to_csv("Animation_movies.csv", index=False)

In [34]:
adventure_movie_df.to_csv("adventure_movies.csv", index=False)

In [ ]:
#list of files
files = ["war_movies_df.csv",
        "action_movies.csv",
        "biography_movies.csv",
        "Animation_movies.csv",
        "adventure_movies.csv"]

#create an empty dataframe to store the combined data
combined_df = []

for file in files:
    com_df = pd.read_csv(file)
    combined_df.append(com_df)
    
combined_df = pd.concat(combined_df, ignore_index = True)
    
    
combined_df.to_csv('IMDB_movies.csv' , index = False)    

In [45]:
df = pd.read_csv('IMDB_movies.csv')

In [46]:
df

,Movie_name,Voting,Rating,Duration,Gener
0,William Tell,1.6,5.8,7980,War
1,The Ministry of Ungentlemanly Warfare,131000.0,6.8,7320,War
2,The Six Triple Eight,25000.0,6.7,7620,War
3,My Dead Friend Zoe,1000.0,7.3,6180,War
4,Nr. 24,14000.0,7.5,6660,War
...,...,...,...,...,...
1723,Elli and the Ghostly Ghost Train,88.0,5.0,5160,Adventure
1724,The Kingfish 2,10.0,7.9,4320,Adventure
1725,Lajjpal,8.0,9.0,15120,Adventure
1726,Beyaz Zenciler: Yapim Asamasi,5.0,8.6,3300,Adventure


In [47]:
! pip install pymysql


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import pymysql 
import pandas as pd

In [ ]:
dbconnection =pymysql.connect(
    host = "localhost",
    database = "mat2",
    user = "mat2",
    password = "0701"
)
curr = dbconnection.cursor()

In [57]:
curr = dbconnection.cursor()

In [62]:
#create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS imdbmovies(
Movie_name VARCHAR(150),
Voting INT,
Rating FLOAT,
Duration INT,
Gener VARCHAR(25)
)
"""

In [63]:
#insert the data into the table
insert_query = """
INSERT INTO imdbmovies(Movie_name, Voting, Rating, Duration, Gener) VALUES (%s, %s, %s, %s, %s)
"""

In [65]:
# Create the table
curr.execute(create_table_query)
dbconnection.commit()

# Insert data into the table
for index, row in df.iterrows():
    try:
        curr.execute(insert_query, (row['Movie_name'], row['Voting'], row['Rating'], row['Duration'], row['Gener']))
    except Exception as e:
        print(f"Error at row {index}: {e}")
dbconnection.commit()


In [66]:
# Run a SELECT query to fetch all rows
select_query = "SELECT * FROM imdbmovies"
curr.execute(select_query)

# Fetch all rows from the result of the query
rows = curr.fetchall()

In [68]:

rows


(('William Tell', 2, 5.8, 7980, 'War'),
 ('The Ministry of Ungentlemanly Warfare', 131000, 6.8, 7320, 'War'),
 ('The Six Triple Eight', 25000, 6.7, 7620, 'War'),
 ('My Dead Friend Zoe', 1000, 7.3, 6180, 'War'),
 ('Nr. 24', 14000, 7.5, 6660, 'War'),
 ('Blitz', 15000, 6.2, 7200, 'War'),
 ('Dirty Angels', 3, 4.3, 6240, 'War'),
 ('Chhava', 2, 8.6, 3060, 'War'),
 ('Amaran', 18000, 8.2, 10020, 'War'),
 ('Uprising', 5, 6.6, 7560, 'War'),
 ('Escape from Germany', 901, 5.3, 5820, 'War'),
 ('Laws of Man', 321, 4.1, 5880, 'War'),
 ('Goebbels and the Führer', 2, 6.7, 8100, 'War'),
 ('Murder Company', 2, 3.7, 5160, 'War'),
 ('Decoded', 410, 5.8, 9360, 'War'),
 ('Dharmarakshak Mahaveer Chhatrapati Sambhaji Maharaj',
  648,
  8.2,
  10440,
  'War'),
 ('Invasion', 664, 4.8, 5460, 'War'),
 ('Before Dawn', 926, 5.4, 6060, 'War'),
 ('Without Blood', 88, 7.5, 5460, 'War'),
 ('Quisling: The Final Days', 1000, 7.1, 8760, 'War'),
 ('Love, Courage and the Battle of Bushy Run', 171, 5.0, 6780, 'War'),
 ('The L